<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
"""
#https://drive.google.com/drive/u/2/my-drive/
dir_id = "1zHVf-O0xXgd47GaVjkpFRUuq59-4Ci5h"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id})
for f in file_list :
  new_list=drive.ListFile({'q': "'%s' in parents and trashed=false" % f[0]["id"]}).GetList()
  a=drive.CreateFile({'id': new_list[0]["id"],'q': "'%s' in parents and trashed=false" % new_list[0]["id"]})
  print(a)
  a.GetContentFile('sample.dcm')
  break
!ls
"""

GoogleDriveFile({'id': '1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh', 'q': "'1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh' in parents and trashed=false"})


FileNotDownloadableError: ignored

In [0]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'    dicom-images-train   手法のアイデア.gdoc
 dicom-images-test   train.csv


In [0]:
!pip install pydicom
!git clone https://github.com/kmt19/siim_otani/
!ls

     |████████████████████████████████| 7.0MB 2.8MB/s 
Cloning into 'siim_otani'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 44 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.
gdrive	sample_data  siim_otani


In [0]:
import os
import pydicom
from glob import glob
from tqdm import tqdm
import pandas as pd
from siim_otani.mask_functions import mask2rle,rle2mask
from siim_otani.pipline2 import get_dataloaders
DRIVE_DIR = 'gdrive/My Drive/'
folders = os.listdir(DRIVE_DIR)
print(folders)


['手法のアイデア.gdoc', 'dicom-images-test', 'dicom-images-train', 'Colab Notebooks', 'train.csv']


In [0]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
print(os.listdir(train_dicom_dir)[0])
a_dir = os.path.join(train_dicom_dir, os.listdir(train_dicom_dir)[0])
print(os.listdir(a_dir))
b_dir = os.path.join(a_dir, os.listdir(a_dir)[0])
print(os.listdir(b_dir))
"""
ここから各Directory内のlistを読み込み、
BASE_DIR = 'train_dicom_dir/%s/%s/' % (file_list, file_list2)を
for文でもらってきて最終的にpathを保存したdataframeを作成する。
"""
for filename in os.listdir(train_dicom_dir):
  BASE_DIR = 'train_dicom_dir/%s/%s/' % (filename, filename)


1.2.276.0.7230010.3.1.2.8323329.13318.1517875244.629616
['1.2.276.0.7230010.3.1.3.8323329.13318.1517875244.629615']
['1.2.276.0.7230010.3.1.4.8323329.13318.1517875244.629617.dcm']


In [0]:
import os
import pandas as pd
from tqdm import tqdm
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import pil_loader
import pydicom
from PIL import Image

class SIIMDataset(Dataset):
    """
    args:
    - phase    : phase of the dataset used (∈{"train", "test"})
    - transform:  
    """
    def __init__(self, phase, df, transform=None):
        self.phase = phase
        #self.root_dir = root_dir
        self.cols = 2 if phase == "train" else 1
        self.df = df #pd.read_csv(f"{root_dir}/{phase}.csv", usecols = [i for i in range(self.cols)])
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx][0]
        print(img_path)
        image = pydicom.read_file(img_path).pixel_array
        image = Image.fromarray(image)
        label = self.df.loc[idx][1] if self.phase == "train" else None
        
        #image = image.convert("RGB")

        if self.transform:
            image = self.transform(image)   # transform image to tensor
        #image = torch.stack(image)
        sample = {'image':image, 'label':label}
        return sample

def get_dataloaders(data, batch_size=8, study_level=False):
    '''
    Returns dataloader pipeline with data augmentation
    '''
    data_transforms = {
        'train': transforms.Compose([
                #transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ]),
        'test': transforms.Compose([
            #transforms.Resize((224, 224)),
            transforms.ToTensor(),
            #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    phase=["train","test"]
    image_datasets = {x:SIIMDataset(x,data[x],transform=data_transforms[x]) for x in phase}
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in phase}
    return dataloaders


In [0]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
#print(len(os.listdir(train_dicom_dir)))
#print(len(os.listdir(test_dicom_dir)))
train_glob = f'{train_dicom_dir}/*/*/*.dcm'
test_glob = f'{test_dicom_dir}/*/*/*.dcm'

exclude_list = []
#train_names = [f for f in tqdm(glob(train_glob))]
#test_names = [f for f in tqdm(glob(test_glob))]


In [24]:
data_files={}
phase = ["train","test"]
data_files["train"] = pd.DataFrame(columns=['Path', 'ID'])
data_files["test"] = pd.DataFrame(columns=['Path'])
SEGMENTATION = DRIVE_DIR + 'train.csv'
anns = pd.read_csv(SEGMENTATION)
train_list=os.listdir(train_dicom_dir)

for i, filename in tqdm(enumerate(train_list)):
  train_glob = f'{train_dicom_dir}/{filename}/*/*.dcm'
  data_files["train"].loc[i] = [glob(train_glob)[0], anns.loc[i][1]]
  if i==10:
    break

test_list=os.listdir(test_dicom_dir)  
for i, filename in tqdm(enumerate(test_list)):
  test_glob = f'{test_dicom_dir}/{filename}/*/*.dcm'
  data_files["test"].loc[i] = [glob(test_glob)[0]]
  if i==10:
    break
traindataloader = get_dataloaders(data_files)
"""
for folder in folders:
  new_folder = path + '/' + folder
  new_folder = new_folder + '/' + os.listdir(new_folder)[0]
  new_file = new_folder + '/' + os.listdir(new_folder)[0]
  ds = pydicom.dcmread(new_file)
  break
  """

0it [00:00, ?it/s]
0it [00:00, ?it/s]


"\nfor folder in folders:\n  new_folder = path + '/' + folder\n  new_folder = new_folder + '/' + os.listdir(new_folder)[0]\n  new_file = new_folder + '/' + os.listdir(new_folder)[0]\n  ds = pydicom.dcmread(new_file)\n  break\n  "

In [26]:
traindataloader = get_dataloaders(data_files,batch_size=1)
for j, data in enumerate(traindataloader["train"]):
  print(data["image"])

g
g
g
g
g
g
g
g
g



KeyboardInterrupt

